In [2]:
from pathlib import Path

LABELS = Path('class_names.txt').read_text().splitlines()

def predict(img):
    x = torch.tensor(img, dtype=torch.float32).unsqueeze(0).unsqueeze(0) / 255.
    with torch.no_grad():
        out = model(x)
    probabilities = torch.nn.functional.softmax(out[0], dim=0)
    values, indices = torch.topk(probabilities, 5)
    confidences = {LABELS[i]: v.item() for i, v in zip(indices, values)}
    return confidences

In [4]:
import torch
from torch import nn

model = nn.Sequential(
    nn.Conv2d(1, 32, 3, padding='same'),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(32, 64, 3, padding='same'),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(64, 128, 3, padding='same'),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Flatten(),
    nn.Linear(1152, 256),
    nn.ReLU(),
    nn.Linear(256, len(LABELS)),
)
state_dict = torch.load('pytorch_model (1).bin',    map_location='cpu')
model.load_state_dict(state_dict, strict=False)
model.eval()

Sequential(
  (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (4): ReLU()
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (7): ReLU()
  (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (9): Flatten(start_dim=1, end_dim=-1)
  (10): Linear(in_features=1152, out_features=256, bias=True)
  (11): ReLU()
  (12): Linear(in_features=256, out_features=100, bias=True)
)

In [5]:
import gradio as gr

gr.Interface(fn=predict, 
             inputs="sketchpad",
             outputs="label",
             title="Mesut Unutur",
             description="Google'ın QUICK DRAW dataseti ile hazırlandı. El ile çizilen objelerden oluşan"+
             "bu datasetin önceden eğitilmiş modeli kullanıldı. Gradio ile servis haline getirildi",
             live=True).launch(share=True)


IMPORTANT: You are using gradio version 3.1.4, however version 3.14.0 is available, please upgrade.
--------
Hint: Set streaming=True for Sketchpad component to use live streaming.
Running on local URL:  http://127.0.0.1:7861/
Running on public URL: https://9fc969a16af0a17f.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://www.huggingface.co/spaces


(<gradio.routes.App at 0x18ebf55e190>,
 'http://127.0.0.1:7861/',
 'https://9fc969a16af0a17f.gradio.app')